<a id='home'></a>
### Содержание

[Imports](#Imports)

[Создание данных](#Созданиеданных)

[Группировки](#Группировки)

[Оконные функции](#Оконныефункции)

[Джоины](#Джоины)

[Cases](#Cases)

   - [Case1](#Case1)
   
   - [Case2](#Case2)

   - [Case3](#Case3)

<a id='Import'></a>
### Imports
[к содержанию](#home)

In [ ]:
import os
import sys
import time
import itertools
from tqdm.notebook import tqdm
import pickle
import json
import joblib
import collections

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import clear_output, display

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

In [ ]:
# # Подключение данных гугл диска
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
!bash /content/drive/MyDrive/spark_colab/spark_colab.sh
# clear_output()

bash: /content/drive/MyDrive/spark_colab/spark_colab.sh: No such file or directory
CPU times: user 10.3 ms, sys: 7 µs, total: 10.3 ms
Wall time: 106 ms


In [ ]:
# Подключаем спарк
#!bash /content/drive/MyDrive/spark_colab/spark_colab.sh
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['SPARK_HOME'] = "/content/drive/MyDrive/spark-3.0.0-bin-hadoop3.2"
# os.environ['SPARK_HOME'] = "/content/drive/MyDrive/ds_interview/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
spark = (SparkSession.builder\
#         .master("local")\
#         .appName("Colab")\
#         .config('spark.ui.port', '4050')\
        .getOrCreate())
spark

---

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

<a id='Созданиеданных'></a>
### Создание данных
[к содержанию](#home)

In [ ]:
# Создадим tmp таблицу и на её примере будет смотреть синтаксис
N = 10_000
np.random.seed(42)
date_range = [pd.Timestamp('2020-01-01') + pd.DateOffset(days=i) for i in range(100)]
tmp_sdf = spark.createDataFrame(pd.DataFrame({
    "group1": np.random.choice(['a', 'b', 'c'], p=[0.2, 0.5, 0.3], size=N),
    "group2": np.random.choice(['gr1', 'gr2', 'gr3', 'gr4'], p=[0.2, 0.2, 0.2, 0.4], size=N),
    "date_col": np.random.choice(date_range, size=N),
    "val": np.random.randint(0, 10, size=N),
}))
# кастуем date_col в тип date
tmp_sdf = tmp_sdf.withColumn('date_col', F.col('date_col').cast('date'))
tmp_sdf = tmp_sdf.orderBy(F.col('group1').asc(), F.col('group2').asc(), F.col('date_col').asc()).cache()
print(f"{tmp_sdf.count():,} x {len(tmp_sdf.columns):,}")

10,000 x 4


In [ ]:
tmp_sdf.show(50)

+------+------+----------+---+
|group1|group2|  date_col|val|
+------+------+----------+---+
|     a|   gr1|2020-01-01|  7|
|     a|   gr1|2020-01-01|  8|
|     a|   gr1|2020-01-01|  9|
|     a|   gr1|2020-01-01|  8|
|     a|   gr1|2020-01-02|  0|
|     a|   gr1|2020-01-02|  6|
|     a|   gr1|2020-01-02|  4|
|     a|   gr1|2020-01-02|  3|
|     a|   gr1|2020-01-02|  3|
|     a|   gr1|2020-01-03|  6|
|     a|   gr1|2020-01-03|  1|
|     a|   gr1|2020-01-03|  1|
|     a|   gr1|2020-01-04|  6|
|     a|   gr1|2020-01-04|  6|
|     a|   gr1|2020-01-04|  7|
|     a|   gr1|2020-01-04|  9|
|     a|   gr1|2020-01-05|  7|
|     a|   gr1|2020-01-05|  6|
|     a|   gr1|2020-01-05|  6|
|     a|   gr1|2020-01-05|  3|
|     a|   gr1|2020-01-05|  6|
|     a|   gr1|2020-01-06|  8|
|     a|   gr1|2020-01-06|  9|
|     a|   gr1|2020-01-06|  0|
|     a|   gr1|2020-01-07|  8|
|     a|   gr1|2020-01-07|  5|
|     a|   gr1|2020-01-08|  5|
|     a|   gr1|2020-01-08|  1|
|     a|   gr1|2020-01-08|  6|
|     a|

In [ ]:
tmp_unq_sdf = (
    tmp_sdf
    .groupBy('group1', 'group2', 'date_col')
    .agg(F.sum(F.col('val')).alias('val'))
    .orderBy(
        F.col('group1').asc(),
        F.col('group2').asc(),
        F.col('date_col').asc()
    )
    .cache()
)
print(f"{tmp_unq_sdf.count():,}")

1,191


In [ ]:
tmp_unq_sdf.show(50)

+------+------+----------+---+
|group1|group2|  date_col|val|
+------+------+----------+---+
|     a|   gr1|2020-01-01| 32|
|     a|   gr1|2020-01-02| 16|
|     a|   gr1|2020-01-03|  8|
|     a|   gr1|2020-01-04| 28|
|     a|   gr1|2020-01-05| 28|
|     a|   gr1|2020-01-06| 17|
|     a|   gr1|2020-01-07| 13|
|     a|   gr1|2020-01-08| 24|
|     a|   gr1|2020-01-09|  0|
|     a|   gr1|2020-01-10|  4|
|     a|   gr1|2020-01-11| 14|
|     a|   gr1|2020-01-12|  7|
|     a|   gr1|2020-01-13| 26|
|     a|   gr1|2020-01-14|  8|
|     a|   gr1|2020-01-15| 15|
|     a|   gr1|2020-01-16| 18|
|     a|   gr1|2020-01-17| 26|
|     a|   gr1|2020-01-18| 32|
|     a|   gr1|2020-01-19| 28|
|     a|   gr1|2020-01-20| 18|
|     a|   gr1|2020-01-21| 20|
|     a|   gr1|2020-01-22| 17|
|     a|   gr1|2020-01-23| 25|
|     a|   gr1|2020-01-24| 30|
|     a|   gr1|2020-01-25| 10|
|     a|   gr1|2020-01-26| 22|
|     a|   gr1|2020-01-27| 29|
|     a|   gr1|2020-01-28| 12|
|     a|   gr1|2020-01-29|  2|
|     a|

In [ ]:
# посмотрим на число уникальных значений в каждом столбце
for col in tmp_unq_sdf.columns:
    cnt = tmp_unq_sdf.select(col).distinct().count()
    print(f"{col}: {cnt:,}")

group1: 3
group2: 4
date_col: 100
val: 127


<a id='Группировки'></a>
### Группировки
[к содержанию](#home)

In [ ]:
# Посмотрим статистики по группе1
(
    tmp_unq_sdf
    .groupBy('group1')
    .agg(
        F.min(F.col('date_col')).alias('min_date'),
        F.max(F.col('date_col')).alias('max_date'),
        F.countDistinct(F.col('group2')).alias('unq_gr2_cnt'),
        F.round(F.mean(F.col('val')), 3).alias('mean_val')
    )
    .orderBy(
        F.col('group1').asc()
    )
    .show()
)

+------+----------+----------+-----------+--------+
|group1|  min_date|  max_date|unq_gr2_cnt|mean_val|
+------+----------+----------+-----------+--------+
|     a|2020-01-01|2020-04-09|          4|  23.641|
|     b|2020-01-01|2020-04-09|          4|  58.248|
|     c|2020-01-01|2020-04-09|          4|  33.314|
+------+----------+----------+-----------+--------+



In [ ]:
# Посмотрим статистики по (группе1, группе2)
(
    tmp_unq_sdf
    .groupBy('group1', 'group2')
    .agg(
        F.min(F.col('date_col')).alias('min_date'),
        F.max(F.col('date_col')).alias('max_date'),
        F.round(F.mean(F.col('val')), 3).alias('mean_val')
    )
    .orderBy(
        F.col('group1').asc(),
        F.col('group2').desc()
    )
    .show()
)

+------+------+----------+----------+--------+
|group1|group2|  min_date|  max_date|mean_val|
+------+------+----------+----------+--------+
|     a|   gr4|2020-01-01|2020-04-09|   38.23|
|     a|   gr3|2020-01-01|2020-04-09|  18.928|
|     a|   gr2|2020-01-01|2020-04-09|  18.392|
|     a|   gr1|2020-01-01|2020-04-09|  18.667|
|     b|   gr4|2020-01-01|2020-04-09|    94.6|
|     b|   gr3|2020-01-01|2020-04-09|   46.65|
|     b|   gr2|2020-01-01|2020-04-09|   45.96|
|     b|   gr1|2020-01-01|2020-04-09|   45.78|
|     c|   gr4|2020-01-01|2020-04-09|   53.29|
|     c|   gr3|2020-01-01|2020-04-09|  27.535|
|     c|   gr2|2020-01-01|2020-04-09|   25.85|
|     c|   gr1|2020-01-01|2020-04-09|  26.455|
+------+------+----------+----------+--------+



<a id='Оконныефункции'></a>
### Оконные функции
[к содержанию](#home)

In [ ]:
# Оконные функции
(
    tmp_unq_sdf
    .withColumn(
        'date_col_n',
        F.row_number().over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
        )
    )
    .withColumn(
        'lag1',
        F.lag(F.col('val'), 1).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
        )
    )
    .withColumn(
        'lag3',
        F.lag(F.col('val'), 3).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
        )
    )
    .withColumn(
        'lag(-1)',
        F.lag(F.col('val'), -1).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
        )
    )
    .withColumn(
        'lst1',
        F.collect_list(F.col('val')).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
            .rowsBetween(W.currentRow-1, W.currentRow)
        )
    )
    .withColumn(
        'lst1_sum',
        F.sum(F.col('val')).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
            .rowsBetween(W.currentRow-1, W.currentRow)
        )
    )
    .withColumn(
        'lst2',
        F.collect_list(F.col('val')).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
            .rowsBetween(W.currentRow-2, W.currentRow+1)
        )
    )
    .withColumn(
        'lst2_max',
        F.max(F.col('val')).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('date_col').asc())
            .rowsBetween(W.currentRow-2, W.currentRow+1)
        )
    )
    # Берём все значения val в промежутке (val-8, val+5) в разрезе (group1, group2)
    .withColumn(
        'lst3',
        F.collect_list(F.col('val')).over(
            W.partitionBy('group1', 'group2').orderBy(F.col('val').asc())
            .rangeBetween(W.currentRow-2, W.currentRow+1)
        )
    )
    .limit(50)
    .toPandas()
)

,group1,group2,date_col,val,date_col_n,lag1,lag3,lag(-1),lst1,lst1_sum,lst2,lst2_max,lst3
0,c,gr4,2020-04-09,13,100,50,48.0,NaN,"[50, 13]",63,"[15, 50, 13]",50,[13]
1,c,gr4,2020-04-07,15,98,48,56.0,50.0,"[48, 15]",63,"[70, 48, 15, 50]",70,"[13, 15]"
2,c,gr4,2020-03-20,19,80,61,46.0,50.0,"[61, 19]",80,"[34, 61, 19, 50]",61,[19]
3,c,gr4,2020-02-19,23,50,53,55.0,41.0,"[53, 23]",76,"[56, 53, 23, 41]",56,[23]
4,c,gr4,2020-01-14,26,14,70,51.0,52.0,"[70, 26]",96,"[83, 70, 26, 52]",83,"[26, 26, 26, 26]"
5,c,gr4,2020-02-01,26,32,35,47.0,39.0,"[35, 26]",61,"[56, 35, 26, 39]",56,"[26, 26, 26, 26]"
6,c,gr4,2020-02-24,26,55,46,90.0,73.0,"[46, 26]",72,"[69, 46, 26, 73]",73,"[26, 26, 26, 26]"
7,c,gr4,2020-03-01,26,61,69,44.0,40.0,"[69, 26]",95,"[70, 69, 26, 40]",70,"[26, 26, 26, 26]"
8,c,gr4,2020-01-07,28,7,71,51.0,47.0,"[71, 28]",99,"[47, 71, 28, 47]",71,"[26, 26, 26, 26, 28, 28, 29, 29, 29]"
9,c,gr4,2020-03-25,28,85,54,57.0,77.0,"[54, 28]",82,"[62, 54, 28, 77]",77,"[26, 26, 26, 26, 28, 28, 29, 29, 29]"


<a id='Джоины'></a>
### Джоины
[к содержанию](#home)

In [ ]:
tmp2_sdf = spark.createDataFrame(pd.DataFrame({
    "group1": ['a', 'b', 'c'],
    "group2": ['gr1', 'gr2', 'gr3'],
    "city": np.random.choice(['Moscow', 'St. P'], size=3)
}))
tmp2_sdf.show()

+------+------+------+
|group1|group2|  city|
+------+------+------+
|     a|   gr1|Moscow|
|     b|   gr2| St. P|
|     c|   gr3| St. P|
+------+------+------+



In [ ]:
joined_sdf = (
    tmp_unq_sdf
    .join(
        tmp2_sdf,
        on=['group1', 'group2'],
        how='left'
    )
)
(
    joined_sdf
    .drop_duplicates(['group1', 'group2'])
    .orderBy('group1', 'group2')
    .show()
)

+------+------+----------+---+------+
|group1|group2|  date_col|val|  city|
+------+------+----------+---+------+
|     a|   gr1|2020-01-01| 32|Moscow|
|     a|   gr2|2020-01-01| 34|  null|
|     a|   gr3|2020-01-01| 23|  null|
|     a|   gr4|2020-01-01| 26|  null|
|     b|   gr1|2020-01-01| 29|  null|
|     b|   gr2|2020-01-01| 38| St. P|
|     b|   gr3|2020-01-01| 14|  null|
|     b|   gr4|2020-01-01|128|  null|
|     c|   gr1|2020-01-01| 23|  null|
|     c|   gr2|2020-01-01|  4|  null|
|     c|   gr3|2020-01-01| 33| St. P|
|     c|   gr4|2020-01-01| 75|  null|
+------+------+----------+---+------+



<a id='Cases'></a>
### Cases
[к содержанию](#home)

<a id='Case1'></a>
#### Case1:
[к содержанию](#home)

Last value in month

In [ ]:
tmp_unq_sdf.show()

+------+------+----------+---+
|group1|group2|  date_col|val|
+------+------+----------+---+
|     a|   gr1|2020-01-01| 32|
|     a|   gr1|2020-01-02| 16|
|     a|   gr1|2020-01-03|  8|
|     a|   gr1|2020-01-04| 28|
|     a|   gr1|2020-01-05| 28|
|     a|   gr1|2020-01-06| 17|
|     a|   gr1|2020-01-07| 13|
|     a|   gr1|2020-01-08| 24|
|     a|   gr1|2020-01-09|  0|
|     a|   gr1|2020-01-10|  4|
|     a|   gr1|2020-01-11| 14|
|     a|   gr1|2020-01-12|  7|
|     a|   gr1|2020-01-13| 26|
|     a|   gr1|2020-01-14|  8|
|     a|   gr1|2020-01-15| 15|
|     a|   gr1|2020-01-16| 18|
|     a|   gr1|2020-01-17| 26|
|     a|   gr1|2020-01-18| 32|
|     a|   gr1|2020-01-19| 28|
|     a|   gr1|2020-01-20| 18|
+------+------+----------+---+
only showing top 20 rows



In [ ]:
(
    tmp_unq_sdf
    .withColumn('month', F.date_trunc('month', 'date_col').cast('date'))
    .withColumn(
        'last_val_in_month',
        F.first(F.col('val')).over(
            W
            .partitionBy(F.col('group1'), F.col('group2'), F.col('month'))
            .orderBy(F.col('date_col').desc())
        )
    )
    .orderBy(
        F.col('group1'),
        F.col('group2'),
        F.col('date_col').asc()
    )
    .show(70)
)

+------+------+----------+---+----------+-----------------+
|group1|group2|  date_col|val|     month|last_val_in_month|
+------+------+----------+---+----------+-----------------+
|     a|   gr1|2020-01-01| 32|2020-01-01|               22|
|     a|   gr1|2020-01-02| 16|2020-01-01|               22|
|     a|   gr1|2020-01-03|  8|2020-01-01|               22|
|     a|   gr1|2020-01-04| 28|2020-01-01|               22|
|     a|   gr1|2020-01-05| 28|2020-01-01|               22|
|     a|   gr1|2020-01-06| 17|2020-01-01|               22|
|     a|   gr1|2020-01-07| 13|2020-01-01|               22|
|     a|   gr1|2020-01-08| 24|2020-01-01|               22|
|     a|   gr1|2020-01-09|  0|2020-01-01|               22|
|     a|   gr1|2020-01-10|  4|2020-01-01|               22|
|     a|   gr1|2020-01-11| 14|2020-01-01|               22|
|     a|   gr1|2020-01-12|  7|2020-01-01|               22|
|     a|   gr1|2020-01-13| 26|2020-01-01|               22|
|     a|   gr1|2020-01-14|  8|2020-01-01

<a id='Case2'></a>
#### Case2:
[к содержанию](#home)

Total position (taking page number into account)

In [ ]:
str_ = 'ABCDEFGHIJKLMNOPQRSTYVWXYZabcdefghijklmnopqrstyvwxyz123456789'

In [ ]:
def get_random_id(): return "".join(np.random.permutation([el for el in str_])[:10])

In [ ]:
data = {'session_id': [], 'position': [], 'page': [], 'has_click': []}
for session in range(15):
    pages = np.random.randint(1, 5)
    positions_per_page = np.random.randint(4, 10, size=pages)
    data['session_id'].extend([get_random_id()]*np.sum(positions_per_page))
    data['position'].extend([position for page in range(pages) for position in range(positions_per_page[page])])
    data['page'].extend([page+1 for page in range(pages) for position in range(positions_per_page[page])])
    data['has_click'].extend(list(np.random.choice([0, 1], p=[0.9, 0.1], size=np.sum(positions_per_page))))
data_sdf = spark.createDataFrame(pd.DataFrame(data))
data_sdf = data_sdf.orderBy('session_id', F.col('page').asc(), F.col('position').asc())
data_sdf.show(30)

+----------+--------+----+---------+
|session_id|position|page|has_click|
+----------+--------+----+---------+
|1opJKIy4jx|       0|   1|        0|
|1opJKIy4jx|       1|   1|        0|
|1opJKIy4jx|       2|   1|        0|
|1opJKIy4jx|       3|   1|        0|
|1opJKIy4jx|       4|   1|        0|
|1opJKIy4jx|       5|   1|        0|
|1opJKIy4jx|       6|   1|        0|
|1opJKIy4jx|       7|   1|        0|
|1opJKIy4jx|       8|   1|        0|
|1opJKIy4jx|       0|   2|        0|
|1opJKIy4jx|       1|   2|        1|
|1opJKIy4jx|       2|   2|        0|
|1opJKIy4jx|       3|   2|        0|
|1opJKIy4jx|       4|   2|        0|
|1opJKIy4jx|       0|   3|        0|
|1opJKIy4jx|       1|   3|        0|
|1opJKIy4jx|       2|   3|        0|
|1opJKIy4jx|       3|   3|        0|
|1opJKIy4jx|       4|   3|        0|
|74Dfvtyypq|       0|   1|        0|
|74Dfvtyypq|       1|   1|        0|
|74Dfvtyypq|       2|   1|        1|
|74Dfvtyypq|       3|   1|        0|
|74Dfvtyypq|       4|   1|        0|
|

In [ ]:
n_positions_per_page_sdf = (
    data_sdf
    .groupBy('session_id', 'page')
    .agg((F.max(F.col('position')) + F.lit(1)).alias('n_positions_per_page'))
    .withColumn(
        'total_positions',
        F.sum(F.col('n_positions_per_page')).over(
            W.partitionBy(F.col('session_id')).orderBy(F.col('page').asc())
        )
    )
    .withColumn(
        'total_positions_before_this_page',
        F.lag(F.col('total_positions')).over(
            W.partitionBy(F.col('session_id')).orderBy(F.col('page').asc())
        )
    )
    .withColumn(
        'total_positions_before_this_page',
        F.when(F.isnull(F.col('total_positions_before_this_page')), F.lit(0))
        .otherwise(F.col('total_positions_before_this_page'))
    )
    .orderBy(F.col('session_id'), F.col('page').asc())
)
n_positions_per_page_sdf.show(20)

+----------+----+--------------------+---------------+--------------------------------+
|session_id|page|n_positions_per_page|total_positions|total_positions_before_this_page|
+----------+----+--------------------+---------------+--------------------------------+
|3FzBaqbi6T|   1|                   9|              9|                               0|
|3FzBaqbi6T|   2|                   4|             13|                               9|
|3FzBaqbi6T|   3|                   9|             22|                              13|
|3FzBaqbi6T|   4|                   4|             26|                              22|
|AdTf4NiFr8|   1|                   4|              4|                               0|
|FK2RsP7J3Y|   1|                   7|              7|                               0|
|IqJyNYw21v|   1|                   8|              8|                               0|
|IqJyNYw21v|   2|                   8|             16|                               8|
|IqJyNYw21v|   3|               

In [ ]:
tmp_sdf = (
    data_sdf
    .join(
        n_positions_per_page_sdf
        .select('session_id', 'page', 'total_positions_before_this_page')
        ,
        on=['session_id', 'page'],
        how='left',
    )
    .withColumn(
        'absolute_position',
        F.col('position') + F.col('total_positions_before_this_page')
    )
    .select('session_id', 'page', 'position', 'absolute_position', 'has_click')
    .orderBy('session_id', F.col('page').asc(), F.col('position').asc())
)

tmp_sdf.limit(50).toPandas()

,session_id,page,position,absolute_position,has_click
0,3FzBaqbi6T,1,0,0,0
1,3FzBaqbi6T,1,1,1,0
2,3FzBaqbi6T,1,2,2,0
3,3FzBaqbi6T,1,3,3,0
4,3FzBaqbi6T,1,4,4,0
5,3FzBaqbi6T,1,5,5,0
6,3FzBaqbi6T,1,6,6,0
7,3FzBaqbi6T,1,7,7,0
8,3FzBaqbi6T,1,8,8,0
9,3FzBaqbi6T,2,0,9,0


In [ ]:
# with row_number:
(
    data_sdf
    .withColumn(
        'absolute_position',
        F.row_number().over(
            W.partitionBy(F.col('session_id'))
            .orderBy(F.col('page').asc(), F.col('position').asc())
        )
    )
    .withColumn('absolute_position', F.col('absolute_position') - F.lit(1))
    .select('session_id', 'page', 'position', 'absolute_position', 'has_click')
    .orderBy('session_id', F.col('page').asc(), F.col('position').asc())
    .show(50)
)

+----------+----+--------+-----------------+---------+
|session_id|page|position|absolute_position|has_click|
+----------+----+--------+-----------------+---------+
|1opJKIy4jx|   1|       0|                0|        0|
|1opJKIy4jx|   1|       1|                1|        0|
|1opJKIy4jx|   1|       2|                2|        0|
|1opJKIy4jx|   1|       3|                3|        0|
|1opJKIy4jx|   1|       4|                4|        0|
|1opJKIy4jx|   1|       5|                5|        0|
|1opJKIy4jx|   1|       6|                6|        0|
|1opJKIy4jx|   1|       7|                7|        0|
|1opJKIy4jx|   1|       8|                8|        0|
|1opJKIy4jx|   2|       0|                9|        0|
|1opJKIy4jx|   2|       1|               10|        1|
|1opJKIy4jx|   2|       2|               11|        0|
|1opJKIy4jx|   2|       3|               12|        0|
|1opJKIy4jx|   2|       4|               13|        0|
|1opJKIy4jx|   3|       0|               14|        0|
|1opJKIy4j



```
# Выбран кодовый формат
```

<a id='Case3'></a>
#### Case3:
[к содержанию](#home)

Explode by column of MapType

In [ ]:
from pyspark.sql import types as T
tmp_sdf = spark.createDataFrame(pd.DataFrame({
    'map_col': [
        {"a": ['a1', "a2", "a3"], "b": ["b1", "b2"], "c": ['c1']},
        {"d": ['d1', "d2"], "e": ["e1", "e2"], "f": ['f1', 'f2', 'f3', 'f4', 'f5']},
        None,
        None
    ]
}))
tmp_sdf = (
    tmp_sdf
    .withColumn('map_col', F.col('map_col').cast(T.MapType(T.StringType(), T.ArrayType(T.StringType()))))
)
print(tmp_sdf.printSchema())
tmp_sdf.toPandas()

/content/drive/MyDrive/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- map_col: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: string (containsNull = true)

None


,map_col
0,"{'a': ['a1', 'a2', 'a3'], 'b': ['b1', 'b2'], '..."
1,"{'d': ['d1', 'd2'], 'e': ['e1', 'e2'], 'f': ['..."
2,None
3,None


In [ ]:
(
    tmp_sdf
    .select(
        F.array(*[
            F.struct(F.lit(key).alias('keys'), F.col('map_col').getItem(key).alias('values'))
            for key in ['a', 'b', 'c', 'd', 'e', 'f']
        ]).alias('map2array')
    )
    .select(F.explode(F.col('map2array')).alias('exploded'))
    .select(
        F.col('exploded')['keys'].alias('keys'),
        F.explode(F.col('exploded')['values']).alias('values')
    )
    .drop_duplicates()
    .groupBy('keys')
    .agg(
        F.countDistinct(F.col('values')).alias('n_unq_values')
    )
    .show()
)

+----+------------+
|keys|n_unq_values|
+----+------------+
|   f|           5|
|   e|           2|
|   d|           2|
|   c|           1|
|   b|           2|
|   a|           3|
+----+------------+

